# Regression to quantify fluorophores

In this notebook, we are using gmodetector_py v0.0.1 to import the data, then will write new code to run regression over it.

## 1. Data import (to be followed by regression itself)

In [1]:
from gmodetector_py import XMatrix
from gmodetector_py import Hypercube

In [2]:
from gmodetector_py import read_wavelengths

In [3]:
import glob

In [4]:
files_list = glob.glob("/scratch2/NSF_GWAS/macroPhor_Array/T16_DEV_genes/EA/wk7/*.hdr")
i = 2
file_path = files_list[i]

In [5]:
wavelengths = read_wavelengths(file_path = file_path)

Header parameter names converted to lower case.


/scratch2/NSF_GWAS/anaconda3/lib/python3.7/site-packages/spectral/io/envi.py:195: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.setttings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


In [6]:
min_desired_wavelength = 500
max_desired_wavelength = 900

In [7]:
test_matrix = XMatrix(fluorophore_ID_vector = ['DsRed', 'ZsYellow', 'Chl', 'Diffraction'],
                      spectral_library_path = "/scratch2/NSF_GWAS/GMOdetectoR/spectra_library/",
                      intercept = 1,
                      wavelengths = wavelengths,
                      spectra_noise_threshold = 0.01,
                      min_desired_wavelength = min_desired_wavelength,
                      max_desired_wavelength = max_desired_wavelength)

0
  wavelength  intensity
0   399.8645   0.000201
1   401.0927   0.000201
2   402.3212   0.000201
3   403.5499   0.000201
4   404.7787   0.000201
0.01
  wavelength  intensity
0   399.8645        0.0
1   401.0927        0.0
2   402.3212        0.0
3   403.5499        0.0
4   404.7787        0.0
1
  wavelength  intensity
0   399.8645        NaN
1   401.0927        NaN
2   402.3212        NaN
3   403.5499        NaN
4   404.7787   0.000706
0.01
  wavelength  intensity
0   399.8645        NaN
1   401.0927        NaN
2   402.3212        NaN
3   403.5499        NaN
4   404.7787        0.0
2
  wavelength  intensity
0   399.8645   0.001076
1   401.0927   0.001076
2   402.3212   0.001076
3   403.5499   0.001076
4   404.7787   0.001076
0.01
  wavelength  intensity
0   399.8645        0.0
1   401.0927        0.0
2   402.3212        0.0
3   403.5499        0.0
4   404.7787        0.0
3
  wavelength  intensity
0   399.8645   0.008441
1   401.0927   0.008441
2   402.3212   0.008441
3   403.5499   0.

In [8]:
from gmodetector_py import find_desired_indices

In [9]:
import numpy as np

In [10]:
subset_indices = find_desired_indices(wavelengths,
                                      min_desired_wavelength,
                                      max_desired_wavelength)
subset_wavelengths = np.array(wavelengths)[subset_indices]

In [11]:
import time

In [12]:
#### Timing of code
time_pre_read = time.perf_counter()
test_cube = Hypercube(file_path,
                      min_desired_wavelength = min_desired_wavelength,
                      max_desired_wavelength = max_desired_wavelength)
time_post_read = time.perf_counter() - time_pre_read
print('Loaded in ' + str(time_post_read) + 's')

Header parameter names converted to lower case.
Header parameter names converted to lower case.


/scratch2/NSF_GWAS/anaconda3/lib/python3.7/site-packages/spectral/io/envi.py:195: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.setttings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


Loaded in 1.5617045620456338s


## 2. Regression (least squares linear algebra approximate solution)

##### In least squares linear regression, we are seeking an approximate solution for weight vector $\vec{m}$ that, when multiplied with design matrix $A$, produces a spectra as close as possible to tr $\vec{b}$

$$A\vec{m} = \vec{b}$$<br>
The design matrix $A$ has a row for each wavelength and a column for each fluorescent protein. The values in this matrix represent the signal intensity of each fluorescent protein at each wavelength.<br>
The vector $\vec{b}$ is a representation of the true spectral signature for a given pixel. This is what we aim to deconvolute into individual components (fluorescent protein signals).<br>
The relative concentrations of individual component are thus indicated by $\vec{m}$, which we will solve for.

The approximate solution for $\vec{m}$ can be derived from the formula for the sum of squared errors (SSE), which we aim to minimize when fitting $\vec{m}$ to our data

##### First, let's look at the formula for squared error for the prediction at a single wavelength $i$.

$$e_i(\vec{m}) = |y_\vec{m}(\vec{a_i})-b_i|^2$$<br>
Here, $e_i(\vec{m})$ is the squared error for the prediction of signal intensity at a single wavelength $i$, with the signal intensity at this wavelength for each fluorescent protein contained in $\vec{a}$

##### Now let's expand this to look at the sum of squared error, $S(\vec{m})$, at all $N$ wavelengths.

$$S(\vec{m}) = \sum_{i=1}^{N}e_i(\vec{m}) = \sum_{i=1}^{N}|y_\vec{m}(\vec{a_i})-b_i|^2)$$<br>
This is simply the summation of the previous formula over all $N$ wavelengths rather than a single wavelength $i$.

##### Let's rewrite this calculus into pure linear algebra and solve for the weight vector $\vec{m}$

$$S(\vec{m}) = ||\vec{y}_\vec{m}-\vec{b}||^2$$
Above, $\vec{y}_\vec{m}$ is a vector of predicted signal intensities (given weights and design matrix) and $\vec{b}$ is a vector of real values for spectral intensity at each wavelength.

$$S(\vec{m}) = ||A\vec{m}-\vec{b}||^2$$
The previous formula is rewritten to reflect that the vector of predicted values $\vec{y}_\vec{m}$ is obtained by matrix-vector multiplication of the weights $\vec{m}$ and design matrix $A$.

If we have a model that perfectly fits the data such that a design matrix $A$ and weight vector $\vec{m}$ have a matrix-vector multiplication product of exactly $\vec{y}_\vec{m}$ then $S(\vec{m})$ will be equal to $0$

Where $A\vec{m} = \vec{b}$<br>
$$S(\vec{m}) = ||A\vec{m}-\vec{b}||^2 = 0$$

However, a perfect fit is not realistic. In practice, we will seek the approximate solution, in which $\vec{m}$ is fit such that $A\vec{m} ≈ \vec{b}$

Where $A\vec{m} \approx \vec{b}$<br>
$$\min_\vec{m}S(\vec{m}) = ||A\vec{m}-\vec{b}||^2$$

##### Linear algebra can then be applied to solve for $\vec{m}$ in a manner that is very computationally efficient.

$$A^TA\vec{m} \approx A^T\vec{b}$$
$$\vec{m} \approx (A^TA)^{-1}A^T\vec{b}$$
$$\vec{m} \approx A^+\vec{b}$$
Note: the matrix $A+ = (A^TA)^{-1}A^T$ is the Moore-Penrose pseudoinverse of $A$

Let's implement this calculation of $\vec{m}$

In [13]:
import numpy as np

In [14]:
test_matrix.matrix.shape

(237, 5)

In [15]:
MP_pseudoinverse = np.linalg.pinv(test_matrix.matrix)

Note that our 3D hypercube needs to be reshaped into a 2D matrix with rows for each $i$ band/wavelength and columns for each $j$ pixel

In [16]:
test_cube.hypercube.shape

(1571, 1419, 237)

In [17]:
b_vec = test_cube.hypercube.reshape(test_cube.hypercube.shape[2],
                                    test_cube.hypercube.shape[0] * test_cube.hypercube.shape[1])

In [18]:
b_vec.shape

(237, 2229249)

Let's make sure that this conversion is reversible.

In [19]:
test_cube.hypercube.shape[0:3]

(1571, 1419, 237)

In [20]:
reconstructed_hypercube = b_vec.reshape(test_cube.hypercube.shape[0:3])

In [21]:
reconstructed_hypercube.all() == test_cube.hypercube.all()

True

##### We are now ready to calculate $\vec{m} \approx A^+\vec{b}$

Note: For multiple linear regression, $\vec{m}$ is a matrix instead of a vector.

In [22]:
MP_pseudoinverse.shape

(5, 237)

In [23]:
m_vec = np.matmul(MP_pseudoinverse, b_vec)

In [24]:
m_vec.shape

(5, 2229249)

##### This needs to be reshaped into a 3D-array of (1571, 1419, 5) where first two dimensions are pixel indices and third is spectral component

In [32]:
m_vec = m_vec.reshape(test_cube.hypercube.shape[0],
                      test_cube.hypercube.shape[1],
                      m_vec.shape[0])

In [33]:
m_vec.shape

(1571, 1419, 5)

Success. Now time to functionalize.

In [51]:
def regress(test_matrix, test_cube):
    """ Perform least squares (multiple) linear regression to regress hypercube onto design matrix and obtain weights for each spectral component in each pixel
    
    :param test_matrix: An object of class XMatrix (containing normalized spectra for each known component, and intercept if applicable)
    :param test_cube: An object of class Hypercube (containing spectra for each pixel)
    """
    
    MP_pseudoinverse = np.linalg.pinv(test_matrix.matrix)
    b_vec = test_cube.hypercube.reshape(test_cube.hypercube.shape[2],
                                        test_cube.hypercube.shape[0] * test_cube.hypercube.shape[1])
    m_vec = np.matmul(MP_pseudoinverse, b_vec)
    m_vec = m_vec.reshape(test_cube.hypercube.shape[0],
                          test_cube.hypercube.shape[1],
                          m_vec.shape[0])
    return(m_vec)

Test and time this function.

In [37]:
#### Timing of code
time_pre_read = time.perf_counter()
weight_array = regress(test_matrix = test_matrix,
                    test_cube = test_cube)
time_post_read = time.perf_counter() - time_pre_read
print('Loaded in ' + str(time_post_read) + 's')

Loaded in 8.609980219043791s


Is the shape correct? Should be (1571, 1419, 5)

In [38]:
weight_array.shape

(1571, 1419, 5)

##### Now make a WeightMatrix class

In [48]:
class WeightArray:
    """A 3D array containing weights for each spectral component, obtained by regression of hybercube onto design matrix
    
    :param test_matrix: An object of class XMatrix (containing normalized spectra for each known component, and intercept if applicable)
    :param test_cube: An object of class Hypercube (containing spectra for each pixel)
    :ivar wavelengths: contains the contents of ``wavelengths`` passed from ``test_matrix`` and ``test_cube`` (must be same, or will yield error)
    :ivar weights: 3D array containing weight values
    
    """
    
    def __init__(self, test_matrix, test_cube):
        
        if test_matrix.wavelengths.astype(np.float).all() != test_cube.wavelengths.astype(np.float).all():
            raise Exception("Design matrix and hypercube do not have the same ``wavelengths`` attribute. Make sure to set the same ``min_desired_wavelength`` and ``max_desired_wavelength`` when creating both objects.")
        
        self.wavelengths = test_matrix.wavelengths
        self.weights = regress(test_matrix = test_matrix,
                               test_cube = test_cube)

In [49]:
#### Timing of code
time_pre_read = time.perf_counter()
weight_array = WeightArray(test_matrix = test_matrix,
                           test_cube = test_cube)
time_post_read = time.perf_counter() - time_pre_read
print('Loaded in ' + str(time_post_read) + 's')

Loaded in 6.021811260841787s


##### As this solution is approximate, it can be useful to calculate the variance and a variance-normalized test-statistic. However, this is very computationally expensive and is NOT done by commercial hyperspectral analysis software LabSpec <a href="https://link.springer.com/article/10.1007/s40789-019-0252-7" target="_blank">Böhme, et al. 2019</a> or KemoQuant.

$$A\vec{m} = \hat{\vec{b}} \approx{\vec{b}}$$
$$\sigma^2 = \frac{S(\vec{m})}{df}$$
$$\sigma^2 = \frac{||A\vec{m}-\vec{b}||^2}{df}$$
$$\sigma^2 = \frac{(A\vec{m}-\vec{b})^T(A\vec{m}-\vec{b})}{df}$$

$$\sigma^2 = \frac{(A\vec{m}-\vec{b})^T(A\vec{m}-\vec{b})}{n-p-1}$$

$\sigma^2$ is the variance of residuals (mean squared error). This is a single value (for a given pixel).

In [25]:
residuals = np.subtract(np.matmul(test_matrix.matrix,
                                  m_vec),
                        b_vec)

In [26]:
residuals.shape

(237, 2229249)

In [27]:
test_matrix.matrix.shape

(237, 5)

In [28]:
time_pre_read = time.perf_counter()
residual_modsqr = np.matmul(residuals.transpose(), residuals)
time_post_read = time.perf_counter() - time_pre_read
print('Calculated in ' + str(time_post_read) + 's')

MemoryError: Unable to allocate 36.2 TiB for an array with shape (2229249, 2229249) and data type float64

Too computationally expensive to do without breaking the matrix down into parts.

$$\widehat{V(\hat{\beta})} = \sigma^2(X^TX)^{-1}$$

$\widehat{V(\hat{\beta})}$ is a matrix containing covariances of linear regression coefficient estimates.

$$\overrightarrow{SE(\hat{\beta})} = diag(\widehat{V(\hat{\beta})})$$

$\overrightarrow{SE(\hat{\beta})}$ is a vector containing the standard variance for each fluorescent protein (preceeded by the intercept, if applicable).

$$\vec{t}=\frac{\hat{\vec{b}}}{\overrightarrow{SE(\hat{\beta})}}$$

$\vec{t}$ is a vector of test statistics, indicating for each component how significant the fluorescent signal is... (preceeded by a term for the y-intercept if applicable)